In [69]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [53]:
df_data = pd.read_csv('../train.csv')
df_data.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [54]:
X = df_data.copy().drop(['id', 'species'],axis='columns').values

species = df_data['species'].unique()

# Création des vecteurs one-hot pour la matrice de targets
t = np.zeros((len(X),species.size)) 
for i in range(species.size):
    t[df_data['species'] == species[i], i] = 1

In [63]:
device = torch.device('cpu')
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.backends.cuda.is_built():
    device = torch.device('cuda')

In [64]:
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.3, random_state=0)
X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
t_train = torch.tensor(t_train, dtype=torch.float32, device=device)
t_test = torch.tensor(t_test, dtype=torch.float32, device=device)

## Création du model

In [65]:
# X.shape[1] : le nombre de caractéristiques
model = nn.Sequential(
    nn.Linear(X.shape[1], 100, bias=True, dtype=torch.float32, device=device), # couche d'entrée
    nn.ReLU(),
    nn.Linear(100, 100, bias=True, dtype=torch.float32, device=device), # couche cachée
    nn.ReLU(),
    nn.Linear(100, species.size, bias=True, dtype=torch.float32, device=device), # couche de sortie
    nn.Softmax(dim=0)
)

In [66]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [68]:
num_epochs = 1000
for epoch in range(num_epochs):
    accuracy = 0
    loss_epoch = 0
    for (x_sample, t_sample) in zip(X_train, t_train): # Dans une epoch nous parcourons toutes les données
        y = model(x_sample)
        loss = loss_function(y, t_sample)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_epoch += loss.item()
        accuracy += (torch.argmax(y) == torch.argmax(t_sample))
    accuracy = accuracy / t_train.shape[0]
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1, num_epochs, loss_epoch, accuracy))

torch.Size([99])
Epoch 1/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 2/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 3/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 4/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 5/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 6/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 7/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 8/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 9/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 10/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 11/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 12/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 13/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 14/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 15/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 16/1000, Loss: 0.000, Accuracy: 0.000
torch.Size([99])
Epoch 17/1000, L

In [13]:
import random
torch.ones(5000000, device=mps_device)  * random.randint(0,100)

tensor([90., 90., 90.,  ..., 90., 90., 90.], device='mps:0')

In [36]:
import random
mps_device = torch.device('cpu')
torch.ones(5000000, device=mps_device)  * random.randint(0,100)

tensor([13., 13., 13.,  ..., 13., 13., 13.])